In [1]:
import torch

In [2]:
class TinyModel(torch.nn.Module):
  def __init__(self):
    super(TinyModel, self).__init__()

    self.linear_1 = torch.nn.Linear(100, 200)
    self.activation = torch.nn.ReLU()
    self.linear_2 = torch.nn.Linear(200, 10)
    self.softmax = torch.nn.Softmax()

  def forward(self, x):
    x = self.linear_1(x)
    x = self.activation(x)
    x = self.linear_2(x)
    x = self.softmax(x)
    return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear_2)

print('\n\nModel params:')
for param in tinymodel.parameters():
  print(param)

print('\n\nLayer params:')
for param in tinymodel.linear_2.parameters():
  print(param)

The model:
TinyModel(
  (linear_1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear_2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[-0.0621, -0.0342,  0.0332,  ..., -0.0087,  0.0863, -0.0318],
        [ 0.0561,  0.0787,  0.0772,  ..., -0.0445,  0.0465,  0.0166],
        [ 0.0121,  0.0049, -0.0004,  ..., -0.0390, -0.0946, -0.0204],
        ...,
        [ 0.0729,  0.0600,  0.0724,  ...,  0.0476, -0.0671,  0.0057],
        [-0.0367,  0.0644,  0.0784,  ..., -0.0509, -0.0932,  0.0748],
        [-0.0961, -0.0209,  0.0741,  ...,  0.0243,  0.0146, -0.0324]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0814, -0.0954,  0.0257,  0.0423,  0.0136,  0.0816, -0.0567,  0.0857,
         0.0109, -0.0855,  0.0264,  0.0687, -0.0996,  0.0607, -0.0070, -0.0277,
         0.0562,  0.0955, -0.0350,  0.

In [3]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
  print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.1586, 0.0267, 0.2324]])


Weight and Bias parameters:
Parameter containing:
tensor([[ 0.3758, -0.0345,  0.1256],
        [-0.5111, -0.0332, -0.5457]], requires_grad=True)
Parameter containing:
tensor([-0.3554,  0.4811], requires_grad=True)


Output:
tensor([[-0.2675,  0.2723]], grad_fn=<AddmmBackward0>)


### Convolutional Layers

In [4]:
import torch.functional as F

In [6]:
class LeNet(torch.nn.Module):

  def __init__(self):
    self.conv_1 = torch.nn.Conv2d(1, 6, 5)
    self.conv_2 = torch.nn.Conv2d(6, 16, 3)

    self.fc_1 = torch.nn.Linear(16*6*6, 120)
    self.fc_2 = torch.nn.Linear(120, 84)
    self.fc_3 = torch.nn.Linear(84, 10)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv_1(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv_2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc_1(x))
    x = F.relu(self.fc_2(x))
    x = self.fc_3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

### Recurrent Layers

In [9]:
"""
vocab_size: number of words in the input vocabulary.
tagset_size: number of tags in the output set.
embedding_dim: size of embedding space for the vocabulary.
hidden_dim: size of LSTM's memory
"""

class LSTMTagger(torch.nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
    super(LSTMTagger, self).__init__()
    self.hidden_dim = hidden_dim

    self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

    # LSTM takes word embeddings as inputs, and outputs hidden states
    # with dimensionality hidden_dim
    self.lstm = torch.nn.LSTM(embedding_dim, tagset_size)

    # Linear layer that maps from hidden state space to tag space
    self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

  def forward(self, sentence):
    embeds = self.word_embeddings(sentence)
    lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
    tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
    tag_scores = F.log_softmax(tag_space, dim=1)
    return tag_scores

### Data Manipulation Layers

In [11]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.6559, 0.9109, 0.2696, 0.6445, 0.4191, 0.6911],
         [0.2124, 0.0079, 0.2259, 0.2805, 0.7323, 0.7055],
         [0.2805, 0.7007, 0.4073, 0.8428, 0.3991, 0.9625],
         [0.4840, 0.5453, 0.7686, 0.7415, 0.5565, 0.5647],
         [0.1706, 0.4429, 0.6901, 0.0588, 0.4640, 0.0817],
         [0.5249, 0.5892, 0.5700, 0.6906, 0.5943, 0.9279]]])
tensor([[[0.9109, 0.9625],
         [0.7686, 0.9279]]])


In [13]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[ 9.2384, 22.8886,  5.0006, 17.7150],
         [12.5224, 23.3427, 23.9775, 17.5678],
         [10.1309, 13.7285, 13.8641,  9.8258],
         [ 8.4485, 14.6576, 23.3204, 14.7525]]])
tensor(15.0613)
tensor([[[-0.6387,  1.3107, -1.2439,  0.5719],
         [-1.4633,  0.8548,  0.9908, -0.3824],
         [-0.9183,  0.9626,  1.0336, -1.0778],
         [-1.2939, -0.1204,  1.5168, -0.1025]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(-4.2841e-08, grad_fn=<MeanBackward0>)


In [14]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.8828, 0.1952, 0.8119, 0.0000],
         [0.0000, 0.3553, 0.7614, 0.0000],
         [0.0657, 0.0000, 0.6914, 0.0000],
         [1.4877, 1.1364, 0.0000, 0.0000]]])
tensor([[[0.8828, 0.1952, 0.0000, 1.0207],
         [0.6426, 0.3553, 0.7614, 0.0000],
         [0.0657, 0.1556, 0.6914, 0.2428],
         [1.4877, 0.0000, 1.6366, 1.1878]]])
